In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats


In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
# exp_names = ["1698157809_Paper_v3_Real30", "1698169547_Paper_v3_Real30_Laplace"]
exp_names = ["1721580789_Real_Laplace_0", "1721403432_Real_opt_seed_fix2"]

exp_comp = "_L"

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += exp_comp
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,RF_large,Platt,ISO,Beta,VA,CT,PPA,Rank,RF_d_L,RF_opt_L,RF_large_L,Platt_L,ISO_L,Beta_L,VA_L,CT_L,PPA_L,Rank_L
Data,,,,,,,,,,,,,,,,,,,,
cm1,0.093479,0.087127,0.087991,0.086781,8.988613e-02,0.087010,0.090390,0.086686,0.086330,0.104688,0.088542,0.086676,0.085832,0.086486,8.969741e-02,8.646474e-02,0.090145,0.085663,0.085602,0.094298
datatrieve,0.086600,0.081560,0.081548,0.080806,8.192088e-02,0.080348,0.092705,0.076917,0.082064,0.086878,0.079490,0.080616,0.077246,0.080373,8.181795e-02,8.218157e-02,0.093265,0.075981,0.080043,0.082110
kc1_class_level_defectiveornot,0.188345,0.179164,0.175868,0.181135,1.896065e-01,0.183354,0.181496,0.190795,0.184349,0.199896,0.180576,0.177063,0.172489,0.180291,1.879750e-01,1.816167e-01,0.180946,0.188997,0.182480,0.199106
kc1,0.110310,0.107667,0.104137,0.110428,1.111313e-01,0.109671,0.110278,0.110476,0.109681,0.114806,0.106778,0.107657,0.103358,0.110198,1.111838e-01,1.090336e-01,0.110259,0.110092,0.109311,0.113959
kc2,0.128132,0.117600,0.121712,0.120711,1.251367e-01,0.123028,0.123603,0.120431,0.121573,0.129083,0.123798,0.117315,0.120376,0.120372,1.252593e-01,1.218321e-01,0.123468,0.119393,0.120365,0.128670
kc3,0.079230,0.076631,0.075165,0.079406,8.509222e-02,0.079528,0.082450,0.078582,0.080551,0.083730,0.076192,0.075980,0.074054,0.079128,8.556298e-02,7.906775e-02,0.082539,0.077187,0.079971,0.083171
pc1,0.054291,0.054253,0.050679,0.056025,5.742756e-02,0.056078,0.057451,0.058515,0.055905,0.059767,0.052162,0.054245,0.050297,0.055801,5.745329e-02,5.579426e-02,0.057392,0.057587,0.055676,0.058047
spect,0.137221,0.123989,0.128855,0.123317,1.279700e-01,0.124080,0.126005,0.122991,0.126363,0.140595,0.130473,0.126171,0.125963,0.122975,1.295926e-01,1.235737e-01,0.126171,0.124239,0.125914,0.142606
spectf,0.139699,0.132407,0.129171,0.137653,1.488664e-01,0.138857,0.141860,0.130364,0.136080,0.144324,0.134853,0.131873,0.128077,0.136255,1.467334e-01,1.368571e-01,0.140251,0.129981,0.135104,0.144283


In [4]:

def stat_test_binary(test_table, method, metric, log=False):

    # Create two sample arrays
    Normal = np.array(test_table.iloc[:, 0])
    Modified = np.array(test_table.iloc[:, 1])

    result = Normal - Modified # for loss comparison
    # result = Modified - Normal # for acc comparison
    if metric == "acc":
        result[result > 0] = 0
        result[result < 0] = 1
    else:
        result[result > 0] = 1
        result[result < 0] = 0
    n_trials = len(result) 
    observed_successes = int(sum(result))

    if log:
        print(f"{metric} {method} observed_successes", observed_successes)

    result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='less')
    p_value = result.pvalue

    if log:
        print("p_value", p_value)

    alpha = 0.05

    if p_value < alpha:
        if log:
            print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
        return 1
    else:
        if log:
            print("Fail to reject the null hypothesis. There is no significant difference.")
        return 0

def stat_test_ttest(test_table, method, metric, log=False):
    print("test_table", test_table)

    if metric == "acc":
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')
    else:
        statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')


    if log:
        print("p_value", p_value)
    alpha = 0.05

    if p_value < alpha:
        if log:
            # print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
            print("The change creates significant improvment")
        return '\ding{51}'
    else:
        if metric == "acc":
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='greater')
        else:
            statistic, p_value = stats.ttest_rel(test_table.iloc[:, 0], test_table.iloc[:, 1], alternative='less')

        if p_value < alpha:
            if log:
                print("The change creates significant decline.")
            return '\ding{55}'
        else:
            if log:    
                print("Fail to reject the null hypothesis. There is no significant difference.")
        return "  "



In [5]:
calib_methods = params["calib_methods"]

metrics =  ["acc", "brier", "logloss", "ece"]

res_table = []
for method in calib_methods:
    res_array = [method]
    for metric in metrics:
        test_table = tables_all[metric][[method, method + exp_comp]]

        # if method == "RF_opt" and metric == "brier":
        #     print(test_table)

        res_array.append(stat_test_ttest(test_table, method, metric))
    res_table.append(res_array)

res_table = np.array(res_table)

res_table = pd.DataFrame(res_table, columns=["calib_methods"]+ metrics)
res_table.set_index('calib_methods', inplace=True)
res_table.to_latex(f"./comp_stat_test.txt")
res_table

test_table                                     RF_d    RF_d_L
Data                                              
cm1                             0.891168  0.890356
datatrieve                      0.906154  0.907692
kc1_class_level_defectiveornot  0.700690  0.715862
kc1                             0.852916  0.854434
kc2                             0.827557  0.831392
kc3                             0.894763  0.895203
pc1                             0.937964  0.937785
spect                           0.814214  0.825493
spectf                          0.784249  0.797778
vertebral                       0.827097  0.825161
wilt                            0.980699  0.980988
parkinsons                      0.887179  0.888205
heart                           0.782874  0.796175
wdbc                            0.954672  0.954339
bank                            0.993441  0.991982
ionosphere                      0.923710  0.929924
HRCompetencyScores              0.912667  0.918000
spambase            

/var/folders/gm/nlbfzh25075c0jwn9khghrfw0000gn/T/ipykernel_64999/2042370032.py:21: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_table.to_latex(f"./comp_stat_test.txt")


,acc,brier,logloss,ece
calib_methods,,,,
RF_d,\ding{51},\ding{51},\ding{51},
RF_opt,,\ding{55},\ding{51},\ding{51}
RF_large,,,\ding{51},\ding{51}
Platt,,\ding{51},\ding{51},
ISO,,,\ding{55},
Beta,\ding{51},\ding{51},\ding{51},
VA,,,,
CT,,,\ding{51},\ding{51}
PPA,,,\ding{51},\ding{51}
